# MVRV 计算方法说明

MVRV（市场价值与已实现价值比率）是一种用于衡量加密资产市场状态的重要指标。
它通过比较市场价值（Market Value, MV）和已实现价值（Realized Value, RV）来判断市场是否被高估或低估。

# 计算方法：
MVRV = 市场价值 / 已实现价值
- 市场价值 (MV)：当前所有流通中的加密资产按照市场价格计算的总价值。
  例如：MV = 当前价格 * 流通供应量
- 已实现价值 (RV)：所有加密资产按照它们最后一次链上移动的价格来计算的总价值。
  RV 反映了市场参与者的平均成本价。
 
 
如果 MVRV > 1，表示当前市场价值高于平均持有成本，市场处于盈利状态，这通常代表有抛售的动力。
如果 MVRV < 1，表示市场价值低于平均持有成本，市场可能被低估，是积累和买入的良好时机。

# 数据来源：
可用`bitquery` 、 `glassnode`来获取链上数据计算，但API使用成本过高。
本例中的数据源自`intotheblock`。

In [9]:
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Line

# 读取 CSV 文件中的数据
df = pd.read_csv('ETH_MVRV.csv')

# 转换数据类型
df['DateTime'] = pd.to_datetime(df['DateTime'])
df['MV Ratio'] = df['MV Ratio'].astype(float)
df['Price'] = df['Price'].astype(float)

# 准备绘图数据
dates = df['DateTime'].dt.strftime("%Y-%m-%d").tolist()
mv_ratios = df['MV Ratio'].tolist()
prices = df['Price'].tolist()

# 相关性计算方法说明

### 什么是相关性

**相关性** 是衡量两个变量之间关系强弱的统计指标，通常用于判断它们是否存在同步变化的趋势。相关性的值介于 **-1** 到 **1** 之间：

- **相关性值为 1**：表示两个变量完全正相关，即当一个变量增加时，另一个变量也以比例关系增加。例如，当市场价格和需求的相关性是 1 时，价格上涨的同时需求也会上涨。
- **相关性值为 -1**：表示两个变量完全负相关，即当一个变量增加时，另一个变量以比例关系减少。例如，当价格和销量的相关性是 -1 时，价格上涨而销量会下降。
- **相关性值为 0**：表示两个变量之间没有线性关系，即它们的变化相互独立。

### 相关性计算公式

相关性通常使用 **皮尔逊相关系数** 来进行计算，公式如下：

$$
 r_{xy} = \frac{\sum{(x_i - \bar{x})(y_i - \bar{y})}}{\sqrt{\sum{(x_i - \bar{x})^2}} \cdot \sqrt{\sum{(y_i - \bar{y})^2}}}
$$

其中：
- $r_{xy}$ 表示变量 $x$ 和 $y$ 之间的相关性。
- $x_i$ 和 $y_i$ 分别是两个变量的每个观测值。
- $\bar{x}$ 和 $\bar{y}$ 分别是两个变量的均值。

### 滚动相关性

在时间序列数据中，通常使用 **滚动相关性** 来观察两个时间序列在某段时间内的关系是如何随时间变化的。滚动相关性通过指定一个 **窗口期**（例如 30 天）来计算这段时间内的相关性，并将这个窗口沿着整个时间序列逐步移动，从而形成一个动态的相关性曲线。

#### 计算滚动相关性的方法：
1. **确定窗口长度**（例如 30 天）。
2. 对于每一个时间点，计算该窗口内的两组数据的相关性。
3. 将相关性值存储下来，形成一条随时间变化的滚动相关性曲线。

In [8]:
# 计算每日增幅百分比
df['price_pct_change'] = df['Price'].pct_change()  # 计算每日价格变化百分比
df['mv_ratio_pct_change'] = df['MV Ratio'].pct_change()  # 计算每日 MVRV 指数变化百分比

# 使用标准化后的数据计算滚动相关性 (30日)
rolling_window = 30
df['rolling_correlation'] = df['price_pct_change'].rolling(window=rolling_window).corr(df['mv_ratio_pct_change'])

# 准备绘图数据
dates = df['DateTime'].dt.strftime("%Y-%m-%d").tolist()
mv_ratios = df['MV Ratio'].tolist()
prices = df['Price'].tolist()
correlation = df['rolling_correlation'].tolist()

# 使用 pyecharts 绘制 ETH MVRV 指数和价格
line = (
    Line(init_opts=opts.InitOpts(bg_color='white'))
    .add_xaxis(dates)
    .add_yaxis(
        "MV Ratio",
        mv_ratios,
        yaxis_index=0,
        linestyle_opts=opts.LineStyleOpts(color="orange"),
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "Price (USD)",
        prices,
        yaxis_index=1,
        linestyle_opts=opts.LineStyleOpts(color="blue"),
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "30-Day Rolling Correlation",
        correlation,
        yaxis_index=2,
        linestyle_opts=opts.LineStyleOpts(color="green"),
        label_opts=opts.LabelOpts(is_show=False)
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="Price (USD)",
            type_="value",
            position="right",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="blue")
            ),
        )
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="30-Day Rolling Correlation",
            type_="value",
            position="right",
            offset=60,  # 偏移量使其与其他轴区分开
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="green")
            ),
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="ETH MVRV Ratio, Price, and 30-Day Rolling Correlation Over Time"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        datazoom_opts=[
            opts.DataZoomOpts(type_="slider", xaxis_index=0),  # 启用时间轴拖动缩放
            opts.DataZoomOpts(type_="inside", xaxis_index=0)   # 启用内嵌缩放
        ],
        yaxis_opts=opts.AxisOpts(
            name="MV Ratio",
            type_="value",
            position="left",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="orange")
            ),
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            name="Date",
            boundary_gap=False,
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(width=2)
            ),
        ),
        legend_opts=opts.LegendOpts(pos_top="10%"),
    )
)

# 显示图表
line.render_notebook()